# plot imgs or feature maps

In [ ]:
import matplotlib.pyplot as plt
import cv2
idx=0
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))  # Adjust figsize as needed

# Display the first image
ax1.imshow(cv2.resize(imgs[idx].permute(1,2,0).detach().cpu().numpy(), (128, 128)))
ax1.set_title('Image 1')
ax1.axis('off')  # Turn off axes for cleaner presentation (optional)

# Display the second image
ax2.imshow(cv2.resize(positive_imgs[idx].permute(1,2,0).detach().cpu().numpy(), (128, 128)))
ax2.set_title('Image 2')
ax2.axis('off')  # Turn off axes (optional)

ax3.imshow(cv2.resize(negative_imgs[idx].permute(1,2,0).detach().cpu().numpy(), (128, 128)))
ax3.set_title('Image 3')
ax3.axis('off')  # Turn off axes (optional)

# Adjust spacing between subplots (optional)
plt.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

# Combine CSV

In [ ]:
import pandas as pd
import os

# Define a list to store the dataframes
all_dfs = []

# Loop through the CSV files and read them into dataframes
for filename in os.listdir('./csv_results/report_class_1/'):  # Replace with your filenames
  df = pd.read_csv(filename)
  all_dfs.append(df)

# Combine all dataframes vertically (stacking rows)
combined_df = pd.concat(all_dfs, ignore_index=True)  # Optional: reset index

# Save the combined dataframe to a new CSV file
combined_df.to_csv("combined.csv", index=False)  # Optional: remove index column


In [ ]:
import pandas as pd
import os
all_dfs = []
names= []
root='./csv_results/report_class_1/'
for filename in os.listdir(root):
    print(filename)
    df = pd.read_csv(root + filename)
    df.insert(loc=0, column='noise', value=filename.split('.')[0])
    all_dfs.append(df)
    names.append(filename.split('.')[0])

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df.to_csv(root + "combined.csv", index=False)

# Create all diffs pickle

In [ ]:
import os
import torch
import pickle
import numpy as np

dataset = 'cifar10'

softmax_sorted = {}
diff_vals = {}
for class_idx in range(10):
    diff_vals[class_idx] = {}

targets_list_loaded = np.load('/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/manual/CIFAR-10-Train-R-A/labels-A.npy')
for class_idx in range(10):
    indices = [k for k in range(len(targets_list_loaded)) if targets_list_loaded[k]==class_idx]
    for file_name in os.listdir('./clip_vec/tensors/'):
        if dataset in file_name and 'target' not in file_name and 'normal_data' not in file_name:
            with open('./clip_vec/tensors/' + file_name, 'rb') as file:
                diffs_loaded = pickle.load(file)
        
            noise_name = file_name.replace(f"{dataset}_diffs_", "", 1).split('.')[0]
            diff_vals[class_idx][noise_name] = torch.mean(torch.tensor([diffs_loaded[idx].astype(np.float32) for idx in indices])).float()

for i in range(10):
    softmaxes = torch.nn.functional.softmax(torch.tensor(list(diff_vals[i].values())), dim=0).numpy()
    for j, key in enumerate(diff_vals[i].keys()):
        diff_vals[i][key] = softmaxes[j]

for i in range(10):
    sorted_values = sorted(diff_vals[i].items(), key=lambda item: item[1])
    diff_vals[i] = dict(sorted_values)


In [ ]:
import os
import torch
import pickle
import numpy as np

dataset = 'svhn'

softmax_sorted = {}
diff_vals = {}
for class_idx in range(10):
    diff_vals[class_idx] = {}

targets_list_loaded = np.load('/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/manual/SVHN-Train-R-A/labels-A.npy')
for class_idx in range(10):
    indices = [k for k in range(len(targets_list_loaded)) if targets_list_loaded[k]==class_idx]
    for file_name in os.listdir('./clip_vec/tensors/'):
        if dataset in file_name and 'target' not in file_name and 'normal_data' not in file_name:
            with open('./clip_vec/tensors/' + file_name, 'rb') as file:
                diffs_loaded = pickle.load(file)
        
            noise_name = file_name.replace(f"{dataset}_diffs_", "", 1).split('.')[0]
            diff_vals[class_idx][noise_name] = torch.mean(torch.tensor([diffs_loaded[idx].astype(np.float32) for idx in indices])).float()

for i in range(10):
    softmaxes = torch.nn.functional.softmax(torch.tensor(list(diff_vals[i].values())), dim=0).numpy()
    for j, key in enumerate(diff_vals[i].keys()):
        diff_vals[i][key] = softmaxes[j]

for i in range(10):
    sorted_values = sorted(diff_vals[i].items(), key=lambda item: item[1])
    diff_vals[i] = dict(sorted_values)


CIFAR100

In [1]:
import os
import torch
import pickle
import numpy as np
CIFAR100_SUPERCLASS = [
        [4, 31, 55, 72, 95],
        [1, 33, 67, 73, 91],
        [54, 62, 70, 82, 92],
        [9, 10, 16, 29, 61],
        [0, 51, 53, 57, 83],
        [22, 25, 40, 86, 87],
        [5, 20, 26, 84, 94],
        [6, 7, 14, 18, 24],
        [3, 42, 43, 88, 97],
        [12, 17, 38, 68, 76],
        [23, 34, 49, 60, 71],
        [15, 19, 21, 32, 39],
        [35, 63, 64, 66, 75],
        [27, 45, 77, 79, 99],
        [2, 11, 36, 46, 98],
        [28, 30, 44, 78, 93],
        [37, 50, 65, 74, 80],
        [47, 52, 56, 59, 96],
        [8, 13, 48, 58, 90],
        [41, 69, 81, 85, 89],
    ]

dataset = 'cifar100'

softmax_sorted = {}
diff_vals = {}
for class_idx in range(20):
    diff_vals[class_idx] = {}

targets_list_loaded = np.load('/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR100_Train_AC/labels_train.npy')
for class_idx in range(20):
    indices = [k for k in range(len(targets_list_loaded)) if targets_list_loaded[k] in CIFAR100_SUPERCLASS[class_idx]]
    for file_name in os.listdir('./clip_vec/tensors_selection/'):
        if dataset in file_name and 'target' not in file_name and 'normal_data' not in file_name:
            print(file_name)
            with open('./clip_vec/tensors_selection/' + file_name, 'rb') as file:
                diffs_loaded = pickle.load(file)
        
            noise_name = file_name.replace(f"{dataset}_diffs_", "", 1).split('.')[0]
            diff_vals[class_idx][noise_name] = torch.mean(torch.tensor([diffs_loaded[idx].astype(np.float32) for idx in indices])).float()

for i in range(20):
    softmaxes = torch.nn.functional.softmax(torch.tensor(list(diff_vals[i].values())), dim=0).numpy()
    for j, key in enumerate(diff_vals[i].keys()):
        diff_vals[i][key] = softmaxes[j]

for i in range(20):
    sorted_values = sorted(diff_vals[i].items(), key=lambda item: item[1])
    diff_vals[i] = dict(sorted_values)


cifar100_diffs_rot90.pkl
cifar100_diffs_random_crop.pkl
cifar100_diffs_jpeg_compression.pkl
cifar100_diffs_gaussian_noise.pkl
cifar100_diffs_glass_blur.pkl
cifar100_diffs_flip.pkl
cifar100_diffs_rot90.pkl
cifar100_diffs_random_crop.pkl
cifar100_diffs_jpeg_compression.pkl
cifar100_diffs_gaussian_noise.pkl
cifar100_diffs_glass_blur.pkl
cifar100_diffs_flip.pkl
cifar100_diffs_rot90.pkl
cifar100_diffs_random_crop.pkl
cifar100_diffs_jpeg_compression.pkl
cifar100_diffs_gaussian_noise.pkl
cifar100_diffs_glass_blur.pkl
cifar100_diffs_flip.pkl
cifar100_diffs_rot90.pkl
cifar100_diffs_random_crop.pkl
cifar100_diffs_jpeg_compression.pkl
cifar100_diffs_gaussian_noise.pkl
cifar100_diffs_glass_blur.pkl
cifar100_diffs_flip.pkl
cifar100_diffs_rot90.pkl
cifar100_diffs_random_crop.pkl
cifar100_diffs_jpeg_compression.pkl
cifar100_diffs_gaussian_noise.pkl
cifar100_diffs_glass_blur.pkl
cifar100_diffs_flip.pkl
cifar100_diffs_rot90.pkl
cifar100_diffs_random_crop.pkl
cifar100_diffs_jpeg_compression.pkl
cifar100

In [3]:
with open('./ranks/selected_noises_cifar100_softmaxed.pkl', 'wb') as f:
    pickle.dump(diff_vals, f)

In [ ]:
import pandas as pd
dfs = []
for i in range(10):
    pd.DataFrame(diff_vals[i], index=[i]).T.to_csv(f'{i}.csv', index=True)

# Cosine similarity noises

In [ ]:

import torch
from PIL import Image
from torchvision import transforms
from clip import clip
from torchvision.datasets import CIFAR10, SVHN
from torch.utils.data import DataLoader
import numpy as np
import PIL
from tqdm import tqdm
import argparse
import pickle
from contrastive import cosine_similarity

class load_np_dataset(torch.utils.data.Dataset):
    def __init__(self, imgs_path, targets_path, transform):
        self.data = np.load(imgs_path)
        self.targets = np.load(targets_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img , target = self.data[idx], self.targets[idx]
            
        img = PIL.Image.fromarray(img)
        if transform:
            img = self.transform(img)

        return img, target



cifar10_path = '/storage/users/makhavan/CSI/finals/datasets/data/'
noises_root = '/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR10_Train_AC/'
cifar_train_cor_target_path = '/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR10_Train_AC/labels_train.npy'
transform = transforms.ToTensor()

noraml_dataset = CIFAR10(root=cifar10_path, train=True, download=True, transform=transform)
normal_loader = DataLoader(noraml_dataset, shuffle=False, batch_size=1)

noise_loaders = {}
for noise in os.listdir(noises_root):
    if noise == 'labels_train.npy':
        continue
    else:
        cifar_train_cor_img_path = noises_root + noise
        aug_dataset = load_np_dataset(cifar_train_cor_img_path, cifar_train_cor_target_path, transform=transform)
        noise_loaders[noise.split('.')[0]] = DataLoader(aug_dataset, shuffle=False, batch_size=1)



cosine_values = {}
for key in noise_loaders.keys():
    cosine_values[key] = []

for key in noise_loaders.keys():
    target_list = []
    for normal_data, noise_data in zip(normal_loader, noise_loaders[key]):
        normal_img, target = normal_data
        noise_img, _ = noise_data
        target_list.append(target)

        cosine_values[key].append(cosine_similarity(normal_img, noise_img).detach().cpu().numpy())

In [ ]:
import pickle
dataset='cifar10'
with open(f'./{dataset}_cosine_softmaxed.pkl', 'rb') as file:
    cosine_values = pickle.load(file)

In [ ]:
import os
import torch
import pickle
import numpy as np


softmax_sorted = {}
diff_vals = {}
for class_idx in range(10):
    diff_vals[class_idx] = {}

for class_idx in range(10):
    indices = [k for k in range(len(target_list)) if target_list[k]==class_idx]
    for key in cosine_values.keys(): 
        # diff_vals[class_idx][key] = np.mean([cosine_values[key][idx].astype(np.float32) for idx in indices])
        diff_vals[class_idx][key] = [cosine_values[key][idx].astype(np.float32) for idx in indices]

for i in range(10):
    softmaxes = torch.nn.functional.softmax(torch.tensor(list(diff_vals[i].values())), dim=0).numpy()
    for j, key in enumerate(diff_vals[i].keys()):
        diff_vals[i][key] = softmaxes[j]

for i in range(10):
    sorted_values = sorted(diff_vals[i].items(), key=lambda item: item[1], reverse=True)
    diff_vals[i] = dict(sorted_values)


# Create 7 Augmentation Image from single one

In [ ]:
from dataset_loader import load_cifar10
import matplotlib.pyplot as plt
import cv2
from torchvision import transforms
from skimage.filters import gaussian
from io import BytesIO
from wand.image import Image as WandImage
from scipy.ndimage import zoom as scizoom
import torch
import numpy as np
import PIL
from PIL import Image
from wand.api import library as wandlibrary

class MotionImage(WandImage):
    def motion_blur(self, radius=0.0, sigma=0.0, angle=0.0):
        wandlibrary.MagickMotionBlurImage(self.wand, radius, sigma, angle)

def rot90(img):
    rotater = transforms.RandomRotation(degrees=(270, 270))
    return rotater(img)


def flip(img):
    hflipper = transforms.RandomHorizontalFlip(p=1.0)
    return hflipper(img)


def random_crop(img):
    resize_cropper = transforms.RandomCrop(size=(480, 480)) # (32, 32)
    resized = transforms.Resize(size=512)(resize_cropper(img))
    return resized


def gaussian_noise(x, severity=1):
    c = [0.04, 0.06, .08, .09, .10][severity - 1]

    x = np.array(x) / 255.
    return np.clip(x + np.random.normal(size=x.shape, scale=c), 0, 1) * 255


def glass_blur(x, severity=1):
    # sigma, max_delta, iterations
    c = [(0.05,1,1), (0.25,1,1), (0.4,1,1), (0.25,1,2), (0.4,1,2)][severity - 1]

    x = np.uint8(gaussian(np.array(x) / 255., sigma=c[0], multichannel=True) * 255)

    # locally shuffle pixels
    for i in range(c[2]):
        for h in range(512 - c[1], c[1], -1):
            for w in range(512 - c[1], c[1], -1):
                dx, dy = np.random.randint(-c[1], c[1], size=(2,))
                h_prime, w_prime = h + dy, w + dx
                # swap
                x[h, w], x[h_prime, w_prime] = x[h_prime, w_prime], x[h, w]

    return np.clip(gaussian(x / 255., sigma=c[0], multichannel=True), 0, 1) * 255


def jpeg_compression(x, severity=1):
    c = [80, 65, 58, 50, 40][severity - 1]

    output = BytesIO()
    x.save(output, 'JPEG', quality=c)
    x = Image.open(output)

    return x


def clipped_zoom(img, zoom_factor):
    h = img.shape[0]
    # ceil crop height(= crop width)
    ch = int(np.ceil(h / zoom_factor))

    top = (h - ch) // 2
    img = scizoom(img[top:top + ch, top:top + ch], (zoom_factor, zoom_factor, 1), order=1)
    # trim off any extra pixels
    trim_top = (img.shape[0] - h) // 2

    return img[trim_top:trim_top + h, trim_top:trim_top + h]


def snow(x, severity=1):
    c = [(0.1,0.2,1,0.6,8,3,0.95),
         (0.1,0.2,1,0.5,10,4,0.9),
         (0.15,0.3,1.75,0.55,10,4,0.9),
         (0.25,0.3,2.25,0.6,12,6,0.85),
         (0.3,0.3,1.25,0.65,14,12,0.8)][severity - 1]

    x = np.array(x, dtype=np.float32) / 255.
    snow_layer = np.random.normal(size=x.shape[:2], loc=c[0], scale=c[1])  # [:2] for monochrome

    snow_layer = clipped_zoom(snow_layer[..., np.newaxis], c[2])
    snow_layer[snow_layer < c[3]] = 0

    snow_layer = Image.fromarray((np.clip(snow_layer.squeeze(), 0, 1) * 255).astype(np.uint8), mode='L')
    output = BytesIO()
    snow_layer.save(output, format='PNG')
    snow_layer = MotionImage(blob=output.getvalue())

    snow_layer.motion_blur(radius=c[4], sigma=c[5], angle=np.random.uniform(-135, -45))

    snow_layer = cv2.imdecode(np.fromstring(snow_layer.make_blob(), np.uint8),
                              cv2.IMREAD_UNCHANGED) / 255.
    snow_layer = snow_layer[..., np.newaxis]

    x = c[6] * x + (1 - c[6]) * np.maximum(x, cv2.cvtColor(x, cv2.COLOR_RGB2GRAY).reshape(512, 512, 1) * 1.5 + 0.5)
    return np.clip(x + snow_layer + np.rot90(snow_layer, k=2), 0, 1) * 255


noises_list = ['rot90', 'flip', 'random_crop', 'gaussian_noise', 'glass_blur', 'jpeg_compression', 'snow']
transform = transforms.ToTensor()
img = transform((cv2.resize(cv2.imread('./figures/airplain_img/airplain.jpg'), (512, 512))))
cv2.imwrite('./figures/car_img/car.jpg', (cv2.resize(cv2.imread('./figures/car_img/car.jpg'), (512, 512))))


In [ ]:
cv2.imwrite('./figures/airplain_img/rot90.jpg', rot90(img).permute(1,2,0).detach().cpu().numpy() * 255.)
cv2.imwrite('./figures/airplain/flip.jpg', flip(img).permute(1,2,0).detach().cpu().numpy() * 255.)
cv2.imwrite('./figures/airplain/random_crop.jpg', random_crop(img).permute(1,2,0).detach().cpu().numpy() * 255.)
cv2.imwrite('./figures/airplain/gaussian_noise.jpg', np.uint8(gaussian_noise(PIL.Image.fromarray((img.permute(1,2,0).detach().cpu().numpy() * 255.).astype(np.uint8)))))
cv2.imwrite('./figures/airplain/glass_blur.jpg', np.uint8(glass_blur(PIL.Image.fromarray((img.permute(1,2,0).detach().cpu().numpy() * 255.).astype(np.uint8)))))
cv2.imwrite('./figures/airplain/jpeg_compression.jpg', np.uint8(jpeg_compression(PIL.Image.fromarray((img.permute(1,2,0).detach().cpu().numpy() * 255.).astype(np.uint8)))))
cv2.imwrite('./figures/airplain/snow.jpg', np.uint8(snow(PIL.Image.fromarray((img.permute(1,2,0).detach().cpu().numpy() * 255.).astype(np.uint8)))))